# Notebook for topic modeling 

# 0. Imports

In [14]:
## load packages 
import pandas as pd
import re
import numpy as np

## nltk imports
#pip install nltk # can install on terminal or by uncommenting this line
import nltk; nltk.download('punkt'); nltk.download('stopwords')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## sklearn imports
from sklearn.feature_extraction.text import CountVectorizer

## lda
#!pip install gensim # can install by uncommenting this line
from gensim import corpora
import gensim

## visualizing LDA--likely need to install
#!pip install pyLDAvis # can install by uncommenting this line
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

## print mult things
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## random
import random
import string; punctlist = [char for char in string.punctuation] # list of english punctuation marks

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellascarola/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellascarola/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 0. Load data

In [66]:
ab = pd.read_csv("../public_data/airbnb_text.zip")
ab.head()



,id,name,name_upper,neighbourhood_group,price
0,2539,Clean & quiet apt home by the park,CLEAN & QUIET APT HOME BY THE PARK,Brooklyn,149
1,2595,Skylit Midtown Castle,SKYLIT MIDTOWN CASTLE,Manhattan,225
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,THE VILLAGE OF HARLEM....NEW YORK !,Manhattan,150
3,3831,Cozy Entire Floor of Brownstone,COZY ENTIRE FLOOR OF BROWNSTONE,Brooklyn,89
4,5022,Entire Apt: Spacious Studio/Loft by central park,ENTIRE APT: SPACIOUS STUDIO/LOFT BY CENTRAL PARK,Manhattan,80


,id,name,name_upper,neighbourhood_group,price
2854,1615764,NaN,NaN,Manhattan,400
3703,2232600,NaN,NaN,Manhattan,200
5775,4209595,NaN,NaN,Manhattan,225
5975,4370230,NaN,NaN,Manhattan,215
6269,4581788,NaN,NaN,Brooklyn,150
6567,4756856,NaN,NaN,Brooklyn,70
6605,4774658,NaN,NaN,Manhattan,40
8841,6782407,NaN,NaN,Brooklyn,45
11963,9325951,NaN,NaN,Manhattan,190
12824,9787590,NaN,NaN,Manhattan,300


# 1. Preprocess documents

In this case, each name/name_upper, or listing title, we're treating as a document

## 1.1 Load stopwords list and augment with our own custom ones

In [24]:
list_stopwords = stopwords.words("english")

list_stopwords

custom_words_toadd = ['apartment', 'new york', 'nyc',
                      'bronx', 'brooklyn',
                     'manhattan', 'queens', 
                      'staten island']

list_stopwords_new = list_stopwords + custom_words_toadd

list_stopwords_new


['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

## 1.2 Remove stopwords from lowercase version of corpus


In [28]:
## convert to lowercase and a list
corpus_lower = ab.name.str.lower().to_list() #first make everything lowercase and put in a list
corpus_lower[0:5]

## use wordpunct tokenize and filter out with one
example_listing = corpus_lower[3]

# use list comprehension to run the tokenizer and remove the stop words including custom words you added
nostop_listing = [word for word in wordpunct_tokenize(example_listing) 
                          if word not in list_stopwords_new]
nostop_listing

['clean & quiet apt home by the park',
 'skylit midtown castle',
 'the village of harlem....new york !',
 'cozy entire floor of brownstone',
 'entire apt: spacious studio/loft by central park']

['cozy', 'entire', 'floor', 'brownstone']

In [ ]:
## can also use .apply instead of making it a list

## 1.3 stem and remove non-alpha

Other contexts we may want to leave digits in

In [30]:
## initialize stemmer
porter = PorterStemmer()

## apply to one by iterating
## over the tokens in the list
example_listing_preprocess = [porter.stem(token) 
                            for token in nostop_listing 
                            if token.isalpha() and 
                            len(token) > 2]

example_listing_preprocess

['cozi', 'entir', 'floor', 'brownston']

In [32]:
example_listing
example_listing_preprocess

'cozy entire floor of brownstone'

['cozi', 'entir', 'floor', 'brownston']

## 1.4 Activity 1

The above example performed preprocessing on a single Airbnb listing. We want to generalize this preprocessing across all listings.

- Embed step two (remove stopwords) and step three (stem) into one or two functions that take in a raw string (eg the raw text of an Airbnb review) and return a preprocessed string 
- Apply the function iteratively to preprocess all the texts in `corpus_lower`. Output could either be a list where each list element is a string of a list (e.g., `cozy brownstone apt`), or a list of lists where each element is a tokenized string (e.g., `['cozy', 'brownstone', 'apt'])`

Output is flexible: it could be a list of lists containing tokenized/stemmed text or a list of strings.

In [104]:
## SETUP
list_stopwords = stopwords.words("english")

custom_words_toadd = ['apartment', 'new york', 'nyc',
                      'bronx', 'brooklyn',
                     'manhattan', 'queens', 
                      'staten island']

list_stopwords_new = list_stopwords + custom_words_toadd
porter = PorterStemmer()

## your code here to define the function(s)
def preprocessing (raw_string):
    ## remove stopwords
    string_token = [word for word in wordpunct_tokenize(raw_string) if word not in list_stopwords_new]

    ## stem
    string_stem = [porter.stem(token) for token in string_token if token.isalpha() and len(token) > 2]

    return string_stem



## your code here to apply the function

#check to see if there are nas bc that is messing up the apply
#ab[ab["name"].isna()] 
ab_no_na = ab[ab["name"].notna()]
corpus_lower_series = ab_no_na.name.str.lower() #leave as a series to use apply

pp_string = corpus_lower_series.apply(preprocessing)
pp_string




## trying to look at it more cleanly
pp_string_df = pp_string.to_frame()

pp_string_df.head()




0                          [clean, quiet, apt, home, park]
1                                 [skylit, midtown, castl]
2                              [villag, harlem, new, york]
3                          [cozi, entir, floor, brownston]
4        [entir, apt, spaciou, studio, loft, central, p...
                               ...                        
48890         [charm, one, bedroom, newli, renov, rowhous]
48891         [afford, room, bushwick, east, williamsburg]
48892                [sunni, studio, histor, neighborhood]
48893                      [time, squar, cozi, singl, bed]
48894               [trendi, duplex, heart, hell, kitchen]
Name: name, Length: 48879, dtype: object

,name
0,"[clean, quiet, apt, home, park]"
1,"[skylit, midtown, castl]"
2,"[villag, harlem, new, york]"
3,"[cozi, entir, floor, brownston]"
4,"[entir, apt, spaciou, studio, loft, central, p..."


# 2. Create a document-term matrix and do some basic diagnostics (more manual approach)

Here we'll create a DTM first using the raw documents; in the activity, you'll create one using the preprocessed docs
that you created in activity 1

## 2.1 Define the dtm function and select data to transform into a document-term matrix

In [112]:
## function provided
def create_dtm(list_of_strings, metadata):
    """ 
    Function to create dense document-term matrix (DTM) from a list of strings and provided metadata. 
    A sparse DTM is a list of term_index/doc_index tuples: if a given term occurs in a given doc at least once, 
        then this count is listed as a tuple; if not, that term/doc pair is omitted. 
    In a dense DTM, each row is one text (e.g., an Airbnb listing), each column is a term, and 
        each cell indicates the frequency of that word in that text. 
    
    Parameters:
        list_of_strings (Series): each row contains a preprocessed string (need not be tokenized)
        metadata (DataFrame): contains document-level covariates
    
    Returns:
        Dense DTM with metadata on left and then one column per word in lexicon
    """
    
    # initialize a sklearn tokenizer; this helps us tokenize the preprocessed string input
    vectorizer = CountVectorizer(lowercase = True) 
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    print('Sparse matrix form:\n', dtm_sparse[:3]) # take a look at sparse representation
    print()
    
    # switch the dataframe from the sparse representation to the normal dense representation (so we can treat it as regular dataframe)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out ())
    print('Dense matrix form:\n', dtm_dense_named.head()) # take a look at dense representation
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1) # add back document-level covariates

    return(dtm_dense_named_withid)

In [114]:

## filter out na's
## for shorter runtime, random sampling of 1000
## get metadata for those
## and also renaming price col since it's likely to be corpus word
ab_small = ab.loc[~ab.name.isnull(),
           ['id', 'neighbourhood_group', 'price', 'name']].copy().rename(columns = {'price':
            'price_rawdata'}).sample(n = 1000, random_state = 422)

ab_small['name_lower'] = ab_small['name'].str.lower()
ab_small.head()

,id,neighbourhood_group,price_rawdata,name,name_lower
23821,19227560,Queens,100,Super Cozy!,super cozy!
22905,18560625,Brooklyn,30,Beautiful Private Bedroom by Prospect Park,beautiful private bedroom by prospect park
20426,16289576,Manhattan,80,Best Location on the Upper West Side! - Part II,best location on the upper west side! - part ii
2018,893413,Manhattan,2500,Architecturally Stunning Former Synagogue!,architecturally stunning former synagogue!
18790,14882137,Queens,50,"Large, beautiful room near Bushwick","large, beautiful room near bushwick"


## 2.2 Execute the dtm function to create the document-term matrix

In [116]:
## example application on raw lowercase texts; 
dtm_nopre = create_dtm(list_of_strings= ab_small.name_lower,
                      metadata = ab_small[['id', 'neighbourhood_group', 'price_rawdata']])



Sparse matrix form:
   (0, 841)	1
  (0, 281)	1
  (1, 152)	1
  (1, 693)	1
  (1, 157)	1
  (1, 205)	1
  (1, 698)	1
  (1, 653)	1
  (2, 165)	1
  (2, 537)	1
  (2, 637)	1
  (2, 856)	1
  (2, 902)	1
  (2, 939)	1
  (2, 774)	1
  (2, 657)	1
  (2, 471)	1

Dense matrix form:
    001  10  10m  10min  10mins  1100  12mins  14  15  15min  ...  yoga  york  \
0    0   0    0      0       0     0       0   0   0      0  ...     0     0   
1    0   0    0      0       0     0       0   0   0      0  ...     0     0   
2    0   0    0      0       0     0       0   0   0      0  ...     0     0   
3    0   0    0      0       0     0       0   0   0      0  ...     0     0   
4    0   0    0      0       0     0       0   0   0      0  ...     0     0   

   you  your  yu  zen  ღღღsteps  法拉盛中心私人房間獨立衛浴  溫馨大套房  獨一無二的紐約閣樓  
0    0     0   0    0         0              0      0          0  
1    0     0   0    0         0              0      0          0  
2    0     0   0    0         0              0      0  

In [118]:
## show first set of rows/cols
dtm_nopre.head()

## show arbitrary later cols in resulting data
dtm_nopre.shape
dtm_nopre.iloc[0:5, 480:500]

,index,id,neighbourhood_group,price_rawdata,001,10,10m,10min,10mins,1100,...,yoga,york,you,your,yu,zen,ღღღsteps,法拉盛中心私人房間獨立衛浴,溫馨大套房,獨一無二的紐約閣樓
0,23821,19227560,Queens,100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22905,18560625,Brooklyn,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20426,16289576,Manhattan,80,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018,893413,Manhattan,2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18790,14882137,Queens,50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(1000, 974)

,inclusive,incredible,incredibly,indoor,inn,inq,insane,int,interior,international,interns,invincible,inviting,inwood,island,it,italy,its,jefferson,jewel
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 2.3 Use that matrix/column sums to get basic summary stats of top words

In [120]:
## summing each col
top_terms = dtm_nopre[dtm_nopre.columns[4:]].sum(axis = 0)

## sorting from most frequent to least frequent
top_terms.sort_values(ascending = False)

in           367
room         244
private      163
bedroom      152
apartment    130
            ... 
gay            1
gente          1
geodesic       1
george         1
獨一無二的紐約閣樓      1
Length: 970, dtype: int64

## 2.4 Activity 2: repeat the above but using the preprocessed text data

- Stick with the same random sample of 1000 `ab_small`
- Apply the preprocessing steps from activity 1 to create a new column in `ab_small` with the preprocessed text (if you got stuck on that, try just removing stopwords)
- Use the `create_dtm` function to create a document-term matrix from the preprocessed data
- Use colsums to summarize

In [144]:
## your code here 

## ab_small (making the data frame)
ab_small = ab.loc[~ab.name.isnull(),
           ['id', 'neighbourhood_group', 'price', 'name']].copy().rename(columns = {'price':
            'price_rawdata'}).sample(n = 1000, random_state = 422)

ab_small['name_lower'] = ab_small['name'].str.lower()
#ab_small.head()


## choosing stopwards and loading how to get stems
list_stopwords = stopwords.words("english")

custom_words_toadd = ['apartment', 'new york', 'nyc',
                      'bronx', 'brooklyn',
                     'manhattan', 'queens', 
                      'staten island']

list_stopwords_new = list_stopwords + custom_words_toadd
porter = PorterStemmer()

## your code here to define the function(s)
def preprocessing (raw_string):
    ## remove stopwords
    string_token = [word for word in wordpunct_tokenize(raw_string) if word not in list_stopwords_new]

    ## stem
    string_stem = [porter.stem(token) for token in string_token if token.isalpha() and len(token) > 2]

    return string_stem



## your code here to apply the function


## wanted to join output
ab_small["name_pp"] = ab_small.name_lower.apply(preprocessing).apply(lambda x: " ".join(x)) ##to have them not be as lists so you can use the create_dtm on them
ab_small.head()


,id,neighbourhood_group,price_rawdata,name,name_lower,name_pp
23821,19227560,Queens,100,Super Cozy!,super cozy!,super cozi
22905,18560625,Brooklyn,30,Beautiful Private Bedroom by Prospect Park,beautiful private bedroom by prospect park,beauti privat bedroom prospect park
20426,16289576,Manhattan,80,Best Location on the Upper West Side! - Part II,best location on the upper west side! - part ii,best locat upper west side part
2018,893413,Manhattan,2500,Architecturally Stunning Former Synagogue!,architecturally stunning former synagogue!,architectur stun former synagogu
18790,14882137,Queens,50,"Large, beautiful room near Bushwick","large, beautiful room near bushwick",larg beauti room near bushwick


In [146]:
## Use the create_dtm function to create a document-term matrix from the preprocessed data

dtm_pp = create_dtm(list_of_strings= ab_small.name_pp,
                      metadata = ab_small[['id', 'neighbourhood_group', 'price_rawdata']])

Sparse matrix form:
   (0, 646)	1
  (0, 170)	1
  (1, 60)	1
  (1, 516)	1
  (1, 65)	1
  (1, 519)	1
  (1, 480)	1
  (2, 70)	1
  (2, 388)	1
  (2, 697)	1
  (2, 729)	1
  (2, 584)	1
  (2, 482)	1

Dense matrix form:
    abcd  abod  access  acidot  acogedor  across  address  ador  aesthet  \
0     0     0       0       0         0       0        0     0        0   
1     0     0       0       0         0       0        0     0        0   
2     0     0       0       0         0       0        0     0        0   
3     0     0       0       0         0       0        0     0        0   
4     0     0       0       0         0       0        0     0        0   

   afford  ...  yard  year  yellow  yoga  york  zen  ღღღstep  法拉盛中心私人房間獨立衛浴  \
0       0  ...     0     0       0     0     0    0        0              0   
1       0  ...     0     0       0     0     0    0        0              0   
2       0  ...     0     0       0     0     0    0        0              0   
3       0  ...     0     

In [148]:
##use columns to summarise

dtm_pp.head()

## show arbitrary later cols in resulting data
dtm_pp.shape
dtm_pp.iloc[0:5, 480:500]

,index,id,neighbourhood_group,price_rawdata,abcd,abod,access,acidot,acogedor,across,...,yard,year,yellow,yoga,york,zen,ღღღstep,法拉盛中心私人房間獨立衛浴,溫馨大套房,獨一無二的紐約閣樓
0,23821,19227560,Queens,100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22905,18560625,Brooklyn,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20426,16289576,Manhattan,80,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018,893413,Manhattan,2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18790,14882137,Queens,50,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(1000, 757)

,pad,palac,para,paradis,park,parkway,part,patio,peac,peach,pendulum,penth,penthous,peopl,perfect,perfectli,person,perstay,pet,photo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 3. Use gensim to more automatically preprocess/estimate a topic model

## 3.1 Creating the objects to feed the LDA modeling function

Different outputs described below: 
- Tokenized and preprocessed text 
- Dictionary 
- Corpus 

In [150]:

## Step 1: re-tokenize and store in list
## here, i'm doing with the raw random sample of text
## in activity, you should do with the preprocessed texts
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  ab_small.name_lower]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, i'm using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so i round
lower_bound = round(ab_small.shape[0]*0.05)
upper_bound = round(ab_small.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### can apply doc2bow(one_text, return_missing = True) to print words
### eliminated from the listing bc they're not in filtered dictionary.
### but feeding that one with missing values to
### the lda function can cause errors
corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]
print('Sample of documents represented in dictionary format (with omitted words noted):')
corpus_fromdict_showmiss[:10]

{0: '!', 1: 'cozy', 2: 'super', 3: 'beautiful', 4: 'bedroom'}

Filtering out very rare and very common words reduced the length of dictionary from 1047 to 31.


{0: '!', 1: 'cozy', 2: 'beautiful', 3: 'bedroom', 4: 'park'}

Sample of documents represented in dictionary format (with omitted words noted):


[([(0, 1), (1, 1)], {'super': 1}),
 ([(2, 1), (3, 1), (4, 1), (5, 1)], {'by': 1, 'prospect': 1}),
 ([(0, 1), (6, 1), (7, 1)],
  {'best': 1,
   'ii': 1,
   'location': 1,
   'on': 1,
   'part': 1,
   'side': 1,
   'upper': 1,
   'west': 1}),
 ([(0, 1)],
  {'architecturally': 1, 'former': 1, 'stunning': 1, 'synagogue': 1}),
 ([(2, 1), (8, 1), (9, 1), (10, 1), (11, 1)], {'bushwick': 1}),
 ([(4, 1), (8, 1), (9, 1), (12, 1), (13, 2)],
  {'bath': 1, 'bed': 1, 'by': 1, 'central': 1, 'college': 1, 'hunter': 1}),
 ([(9, 1), (11, 1), (14, 1), (15, 1)], {'bohemian': 1, 'brownstone': 1}),
 ([(16, 1)],
  {'fidi': 1, 'huge': 1, 'loft': 1, 'views': 1, 'w': 1, 'water': 1}),
 ([], {'hillside': 1, 'hotel': 1}),
 ([(5, 1), (9, 1), (11, 1), (14, 1), (15, 1)], {'airy': 1})]

##  3.2 Estimating the model

In [152]:
## Step 5: we're finally ready to estimate the model!
## full documentation here - https://radimrehurek.com/gensim/models/ldamodel.html
## here, we're feeding the lda function:
## (1) the corpus we created from the dictionary,
## (2) a parameter we decide on for the number of topics (k),
## (3) the dictionary itself,
## (4) parameter for number of passes through training data (more means slower), and
## (5) parameter that returns, for each word remaining in dict, the topic probabilities.
## see documentation for many other arguments you can vary
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 5, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True)

print(type(ldamod))



<class 'gensim.models.ldamodel.LdaModel'>


## 3.3  Seeing what topics the estimated model discovers

In [154]:
## Post-model 1: explore corpus-wide summary of topics
### getting the topics and top words; can retrieve diff top words
topics = ldamod.print_topics(num_words = 10)
for topic in topics:
    print(topic)


(0, '0.311*"!" + 0.075*"." + 0.068*"brooklyn" + 0.066*"/" + 0.065*"and" + 0.059*"," + 0.059*"to" + 0.052*"the" + 0.041*"studio" + 0.032*"cozy"')
(1, '0.159*"apartment" + 0.115*"bedroom" + 0.095*"studio" + 0.080*"1" + 0.074*"east" + 0.071*"2" + 0.054*"beautiful" + 0.052*"-" + 0.051*"large" + 0.042*"with"')
(2, '0.238*"in" + 0.135*"room" + 0.113*"private" + 0.051*"williamsburg" + 0.046*"brooklyn" + 0.046*"spacious" + 0.043*"the" + 0.041*"bedroom" + 0.038*"of" + 0.031*"apt"')
(3, '0.190*"-" + 0.133*"park" + 0.113*"in" + 0.077*"room" + 0.063*"near" + 0.058*"sunny" + 0.043*"cozy" + 0.043*"2" + 0.042*"bedroom" + 0.030*"spacious"')
(4, '0.280*"," + 0.081*"to" + 0.073*"room" + 0.070*"/" + 0.066*"manhattan" + 0.054*"&" + 0.044*"in" + 0.040*"cozy" + 0.031*"large" + 0.029*"near"')


In [156]:
    
## Post-model 2: explore topics associated with each document
### for each item in our original dictionary, get list of topic probabilities
l=[ldamod.get_document_topics(item) for item in corpus_fromdict]
### print result
text_raw_tokens[0:5]
l[0:5]

[['super', 'cozy', '!'],
 ['beautiful', 'private', 'bedroom', 'by', 'prospect', 'park'],
 ['best',
  'location',
  'on',
  'the',
  'upper',
  'west',
  'side',
  '!',
  '-',
  'part',
  'ii'],
 ['architecturally', 'stunning', 'former', 'synagogue', '!'],
 ['large', ',', 'beautiful', 'room', 'near', 'bushwick']]

[[(0, 0.7496185),
  (1, 0.056587867),
  (2, 0.09588674),
  (3, 0.048570838),
  (4, 0.04933604)],
 [(0, 0.026072213),
  (1, 0.033575874),
  (2, 0.10982796),
  (3, 0.80170757),
  (4, 0.02881645)],
 [(0, 0.5305534),
  (1, 0.041816887),
  (2, 0.07074397),
  (3, 0.32056078),
  (4, 0.036325015)],
 [(0, 0.61851865),
  (1, 0.087528884),
  (2, 0.14265949),
  (3, 0.075058855),
  (4, 0.076234154)],
 [(0, 0.02159238),
  (1, 0.02789292),
  (2, 0.047797896),
  (3, 0.023554798),
  (4, 0.87916195)]]

### Visualizing 

In [158]:
lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin

## 3.4 Activity 3

- Preprocess the texts if you haven't already
- Run the topic model with preprocessed texts
- Play around with other parameters like `n_topics` to find a configuration that produces useful topics

If you get stuck on the preprocessing part, you can use below function and example code for applying it. Then continue as above (start with tokenizing).

In [180]:
# your code here

## Step 1: re-tokenize and store in list
## here, i'm doing with the raw random sample of text
## in activity, you should do with the preprocessed texts
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  ab_small.name_pp]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, i'm using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so i round
lower_bound = round(ab_small.shape[0]*0.05)
upper_bound = round(ab_small.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### can apply doc2bow(one_text, return_missing = True) to print words
### but feeding that one with missing values to
### the lda function can cause errors
corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]
print('Sample of documents represented in dictionary format (with omitted words noted):')
corpus_fromdict_showmiss[:10]


## Step 5: we're finally ready to estimate the model!
## full documentation here - https://radimrehurek.com/gensim/models/ldamodel.html
## here, we're feeding the lda function:
## (1) the corpus we created from the dictionary,
## (2) a parameter we decide on for the number of topics (k),
## (3) the dictionary itself,
## (4) parameter for number of passes through training data (more means slower), and
## (5) parameter that returns, for each word remaining in dict, the topic probabilities.
## see documentation for many other arguments you can vary


## IN HERE PLAY AROUIND WIHT NUM TIOPICS AND PASSES
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 4, 
                                         id2word=text_raw_dict, 
                                         passes=10, #increasing the hot and cold idea-->like how nmany questions you would ask (how algorithms convverge)
                                         alpha = 'auto',
                                         per_word_topics = True)

print(type(ldamod))

## Post-model 1: explore corpus-wide summary of topics
### getting the topics and top words; can retrieve diff top words
topics = ldamod.print_topics(num_words = 10)
for topic in topics:
    print(topic)

## Post-model 2: explore topics associated with each document
### for each item in our original dictionary, get list of topic probabilities
l=[ldamod.get_document_topics(item) for item in corpus_fromdict]
### print result
text_raw_tokens[0:5]
l[0:5]

## Display
lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

{0: 'cozi', 1: 'super', 2: 'beauti', 3: 'bedroom', 4: 'park'}

Filtering out very rare and very common words reduced the length of dictionary from 753 to 14.


{0: 'cozi', 1: 'beauti', 2: 'bedroom', 3: 'park', 4: 'privat'}

Sample of documents represented in dictionary format (with omitted words noted):


[([(0, 1)], {'super': 1}),
 ([(1, 1), (2, 1), (3, 1), (4, 1)], {'prospect': 1}),
 ([], {'best': 1, 'locat': 1, 'part': 1, 'side': 1, 'upper': 1, 'west': 1}),
 ([], {'architectur': 1, 'former': 1, 'stun': 1, 'synagogu': 1}),
 ([(1, 1), (5, 1), (6, 1), (7, 1)], {'bushwick': 1}),
 ([(3, 1), (5, 1)],
  {'bath': 1, 'bed': 1, 'central': 1, 'colleg': 1, 'hunter': 1}),
 ([(5, 1), (7, 1)], {'bohemian': 1, 'brownston': 1}),
 ([(8, 1)], {'fidi': 1, 'huge': 1, 'loft': 1, 'view': 1, 'water': 1}),
 ([], {'hillsid': 1, 'hotel': 1}),
 ([(4, 1), (5, 1), (7, 1)], {'airi': 1})]

<class 'gensim.models.ldamodel.LdaModel'>
(0, '0.203*"apt" + 0.193*"park" + 0.170*"larg" + 0.147*"sunni" + 0.142*"near" + 0.085*"room" + 0.049*"beauti" + 0.004*"privat" + 0.002*"bedroom" + 0.001*"studio"')
(1, '0.463*"room" + 0.359*"privat" + 0.133*"williamsburg" + 0.022*"cozi" + 0.007*"bedroom" + 0.006*"spaciou" + 0.005*"east" + 0.001*"sunni" + 0.001*"beauti" + 0.001*"larg"')
(2, '0.373*"studio" + 0.370*"spaciou" + 0.076*"east" + 0.062*"apt" + 0.054*"sunni" + 0.030*"williamsburg" + 0.014*"room" + 0.006*"bedroom" + 0.006*"beauti" + 0.003*"near"')
(3, '0.482*"bedroom" + 0.273*"cozi" + 0.127*"beauti" + 0.100*"east" + 0.005*"near" + 0.002*"studio" + 0.002*"apt" + 0.002*"room" + 0.002*"sunni" + 0.001*"park"')


[['super', 'cozi'],
 ['beauti', 'privat', 'bedroom', 'prospect', 'park'],
 ['best', 'locat', 'upper', 'west', 'side', 'part'],
 ['architectur', 'stun', 'former', 'synagogu'],
 ['larg', 'beauti', 'room', 'near', 'bushwick']]

[[(0, 0.13411795), (1, 0.16383415), (2, 0.10677789), (3, 0.59527004)],
 [(0, 0.28948155), (1, 0.26162603), (2, 0.04450604), (3, 0.4043864)],
 [(0, 0.25171965), (1, 0.30336767), (2, 0.20041662), (3, 0.2444961)],
 [(0, 0.25171965), (1, 0.30336767), (2, 0.20041662), (3, 0.2444961)],
 [(0, 0.78055847), (1, 0.11620485), (2, 0.044564117), (3, 0.05867254)]]

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=18536) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin